In [1]:
# set up the stage
#!pip install cropharvest
from cropharvest.datasets import CropHarvest
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/GitHub/cropharvest

In [12]:
# importing libraries, connecting to gee
import ee
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWgG-qNTW4gTwHfVTbOCTEve2ZYnd4azmIUkndnDvUH9jBITHIhJcoQ

Successfully saved authorization token.


In [13]:
# Visualization function as I am a visual person
import sys
path = '/Documents/GitHub/cropharvest/cropharvest/eo'
# insert at 1, 0 is the script path (or '' in REPL)
%cd cropharvest
%cd eo
%ls
import folium
from visualize import add_ee_layer
from sentinel2_altered import get_single_image_choice, computeS2CloudScore, computeS2CloudScore_l2a, get_single_image_s22a, projectShadows, computeQualityScore

[Errno 2] No such file or directory: 'cropharvest'
/mnt/c/Users/sheil/Documents/GitHub/cropharvest/cropharvest/eo
[Errno 2] No such file or directory: 'eo'
/mnt/c/Users/sheil/Documents/GitHub/cropharvest/cropharvest/eo
__init__.py*     eo.py*         sentinel2.py*          utils.py*
__pycache__/     era5.py*       sentinel2_altered.py*  visualize.py*
classifiers.py*  sentinel1.py*  srtm.py*


In [14]:
# import eo stuff from harvest
#from cropharvest.eo import eo, era5, utils
#from cropharvest.eo.sentinel2 import get_single_image

In [43]:
# import sentinel l1c and 2a
from utils import date_to_string
from datetime import date
start_date = date(2019, 6, 30)
end_date = date(2019, 7,30)
region_us = ee.Geometry.Polygon([ [[-85.23605346679688, 40.06178210817461],\
                                          [-85.06027221679688, 40.06178210817461], \
                                          [-85.06027221679688, 40.196183582024986],\
                                          [-85.23605346679688, 40.196183582024986],\
                                          [-85.23605346679688, 40.06178210817461]]])

# try a small geometry in togo
region = ee.Geometry.Polygon( [[0.9118652343749999,\
              7.634775927004439\
            ],\
            [\
              1.02447509765625,\
              7.634775927004439\
            ],\
            [\
              1.02447509765625,\
              7.7490943810826565\
            ],\
            [\
              0.9118652343749999,\
              7.7490943810826565\
            ],\
            [\
              0.9118652343749999,\
              7.634775927004439\
            ]])


In [31]:

# from before, when I was testing out each step
#start_date = '2019-06-30'
#end_date = '2019-07-30'
s2_l1c = ee.ImageCollection("COPERNICUS/S2").filterDate(start_date, end_date).filterBounds(region);
s2_l2a = ee.ImageCollection("COPERNICUS/S2_SR").filterDate(start_date, end_date).filterBounds(region);


s2_l1c = s2_l1c.map(computeS2CloudScore).map(projectShadows).map(computeQualityScore).sort("CLOUDY_PIXEL_PERCENTAGE").first();
s2_l2a = s2_l2a.map(computeS2CloudScore_l2a).map(projectShadows).map(computeQualityScore).sort("CLOUDY_PIXEL_PERCENTAGE").first();

# cloudScore
s2_l1c_cloudScore = s2_l1c.select('cloudScore');
s2_l2a_cloudScore = s2_l2a.select('cloudScore');

# shadowScore
s2_l1c_shadowScore = s2_l1c.select('shadowScore');
s2_l2a_shadowScore = s2_l2a.select('shadowScore');


# qualityScore

s2_l1c_cloudShadowScore = s2_l1c.select('cloudShadowScore');
s2_l2a_cloudShadowScore = s2_l2a.select('cloudShadowScore');

# difference shadowScore

shadowScore_diff = s2_l2a_shadowScore.subtract(s2_l1c_shadowScore);
#visualization

rgbVis = {'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']};

totalScoreVis = {'min': -1.0, 'max': 1};

absVis = {'min': -1.0 , 'max': 1};




In [69]:
print(s2_l1c_shadowScore.getInfo())

{'type': 'Image', 'bands': [{'id': 'shadowScore', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.0001796630568239043, 0, 0, 0, -0.0001796630568239043, 0]}], 'id': 'COPERNICUS/S2/20190703T101031_20190703T102231_T31NBJ', 'version': 1562231192355578, 'properties': {'DATATAKE_IDENTIFIER': 'GS2A_20190703T101031_021040_N02.07', 'SPACECRAFT_NAME': 'Sentinel-2A', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 101.172136163, 'MEAN_SOLAR_AZIMUTH_ANGLE': 52.7271880391, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[1.2733590423179983, 8.138449959217429], [1.2733440594506205, 8.138452672422869], [0.2775760203448154, 8.132997798958378], [0.27753438515958173, 8.132960991106637], [0.27748845035101316, 8.132929603254755], [0.27748565556721694, 8.132914770715022], [0.2791334661748918, 7.884933993749908], [0.280728599382393, 7.636957453722534], [0.2822712637271953, 7.388977790729161], [0.2837616679940183, 7.140994986286197], [0.28379

In [66]:
start_date = '2019-06-30' #date(2019, 6, 30)
#end_date = '2019-07-30'#date(2019, 7,30)
end_date = '2019-07-15'
s2_l1c = ee.ImageCollection("COPERNICUS/S2").filterDate(start_date, end_date).filterBounds(region);
s2_l2a = ee.ImageCollection("COPERNICUS/S2_SR").filterDate(start_date, end_date).filterBounds(region);



best_l1c = get_single_image_choice(region, start_date, end_date, s2_l1c).clip(region)
best_l2a = get_single_image_s22a(region, start_date, end_date, s2_l2a).clip(region)

#print(best_l2a.getInfo())
#print(best_l1c.getInfo())

In [67]:
# visualize these

rgbVis = {'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

folium.Map.addLayer = add_ee_layer
my_map = folium.Map(region.centroid(maxError =  1).getInfo()['coordinates'], zoom_start=5, height=500)


'''
# from before, testing out each step in the mosaicking
#true color images of Sentinel-2
my_map.addLayer(s2_l1c, rgbVis, 'L1C');
my_map.addLayer(s2_l2a, rgbVis, 'L2A');

#looking at where the clouds are
my_map.addLayer(s2_l1c_cloudScore, absVis, 'L1C cloudScore');
my_map.addLayer(s2_l2a_cloudScore, absVis, 'L2A cloudScore');

#Map.addLayer(s2_l2a_cloudProb , {}, 'L2A cloudProb');

#looking at cloud shadows
my_map.addLayer(s2_l1c_shadowScore, absVis, 'L1C shadowScore');
my_map.addLayer(s2_l2a_shadowScore, absVis, 'L2A shadowScore');

#looking at difference in cloud shadows
my_map.addLayer(shadowScore_diff, absVis, 'difference in shadows');

#looking at total score (cloudShadow)
my_map.addLayer(s2_l1c_cloudShadowScore, absVis, 'L1C cloudShadowScore');
my_map.addLayer(s2_l2a_cloudShadowScore, absVis, 'L2A cloudShadowScore');

'''
# adding the best image from the month
my_map.addLayer(best_l1c, rgbVis, 'L1C Best');
my_map.addLayer(best_l2a, rgbVis, 'L2A Best');

my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [68]:
viz = {'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

best_l1c_url = best_l1c.getThumbUrl({'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2'], 'scale' : 10})
best_l2a_url = best_l2a.getThumbUrl({'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2'], 'scale':  10})
print('L2A:', start_date, end_date, best_l2a_url)
print('L1C:', start_date, end_date, best_l1c_url)
from IPython.display import Image
#Image(url=best_l1c_url, embed=True, format='png')
#Image(url=best_l2a_url, embed=True, format='png')



L2A: 2019-06-30 2019-07-15 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/17e2231951d6feec5e90ddfbb1e69042-6d40c4ca8294df87b2043a925ead8122:getPixels
L1C: 2019-06-30 2019-07-15 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c2d2c83b2e9af0a7776d3dc091652730-efee788e1a63dfd73f488176d4df0e8e:getPixels


In [ ]:
'''
# VISUALIZE
# getting CDL

print(date_to_string(start_date))

### be sure to put this in utils
def get_cdl_year(starting_date):
  '''
  starting_date = string
  returns tuple
  '''
  year = date_to_string(starting_date)[:4]
  newyear = year+'-01-01'
  endyear = year+'-12-31'
  return newyear, endyear

rgbVis = {'min': 0.0, 'max': 1000, 'bands': ['B4', 'B3', 'B2']}
ground_truth = ee.ImageCollection('USDA/NASS/CDL')\
                    .filter(ee.Filter.date(get_cdl_year(start_date)[0], get_cdl_year(start_date)[1]))\
                    .first()\
                    .clip(region)

classvalue = ground_truth.get('cropland_class_values')#; //Type: LIST OF POSSIBLE VALUES
croplandcover = ee.Image(ground_truth.select('cropland'))#; // Type: IMAGE

print(ground_truth.getInfo());
folium.Map.add_ee_layer = add_ee_layer
my_map = folium.Map(location=[45.5236, -122.6750], zoom_start=3, height=500)

my_map.add_ee_layer(croplandcover, {}, 'crop cover')
my_map.add_ee_layer(img_l1c, rgbVis, 'Sentinel-2 L1C mosaic')
my_map.add_ee_layer(img_l2a, rgbVis, 'Sentinel-2 L2A mosaic')


my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)
'''

In [ ]:
## run some classifiers on this thing

def classify_rf(image1, classes, input_region):
  # taking CDL as input

  #fix this so it takes ground_truth directly



  #//trains on image1
  bands = ['B1', 'B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
  label = 'cropland'
  #this has to be a Feature Class, can't be a raster/image
  

  # using a random split method

  sample = image1.addBands(classes.select(label)).sample(numPixels = 50000,
                         seed = 0,
                         region = input_region,
                         geometries = True,
                         scale = 10)
  

  sample = sample.randomColumn();

  split = 0.7 # for training on 70% of the image, testing on 30%
  training = sample.filter(ee.Filter.lt('random', split));
  validation = sample.filter(ee.Filter.gte('random', split));

  print("training size", training.size().getInfo())
  print('validation size', validation.size().getInfo())

  # using the sampleRegions method
  #training = image1.select(bands).sampleRegions(
  #                                              collection = classes,
  #                                              properties = [label],
                                                # geometries = True,
                                                # scale = 30,
                                                # seed = 0
                                                # )      
  #//this has to be a Feature Class, can't be a raster/image
  # validation sourced from same EO image, using different seed to get different points
  # validation = image1.select(bands).sampleRegions(
  #                                                 collection = classes,
  #                                                 properties = [label],
  #                                                 geometries = True,
  #                                                 scale = 30,
  #                                                 seed = 1)#starting with different random seed
  #print('training', training)
  #print('validation', validation)
  # Trained with 70% of our data.
  classifier = ee.Classifier.smileRandomForest(50, 8)\
                              .train(
                                  features = training,
                                  classProperty = label,
                                  inputProperties= bands
                                  );
  classified = image1.classify(classifier)

  trainAccuracy = classifier.confusionMatrix()
  #print('Resubstitution error matrix: ', trainAccuracy.getInfo())
  print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())



  #//we then classify the second image
  
  validated = validation.classify(classifier);



  #print(validated.getInfo(), "validated info");
  testAccuracy = validated.errorMatrix('cropland', 'classification');
  print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

  #print('Validation error matrix: ', testAccuracy.getInfo());
  #print('Validation overall accuracy: ', testAccuracy.accuracy());

  confMatrix = classifier.confusionMatrix();
  OA = confMatrix.accuracy();
  CA = confMatrix.consumersAccuracy();
  Kappa = confMatrix.kappa();
  Order = confMatrix.order();
  PA = confMatrix.producersAccuracy();
  #print("PA", PA.getInfo())
  
  print('Confusion Matrix', confMatrix.getInfo());
  print('Overall Accuracy', OA.getInfo());
  print('Consumers Accuracy', CA.getInfo());
  print('Kappa', Kappa.getInfo());
  #print('Order', Order);
  #print('Producers Accuracy', PA);
  return validated, testAccuracy.accuracy#testAccuracy#;//TYPE: Image 


In [ ]:
print(classify_rf(img_l1c, croplandcover, region))

In [ ]:
print(croplandcover.bandNames().getInfo())

input_region = region
image1 = img
classes = croplandcover
sample = image1.addBands(classes).sample(numPixels = 5000,
                                         seed = 0,
                                         region = input_region,
                                         geometries = True,
                                         scale = 30)

print(sample.getInfo())